# I have crated a markdown to replicate my CUDA-fitting problem, where I load pre-embedded reviews, to make loading time shorter.

In [1]:
# Load libraries
!pip install -r /work/NLP_IMDb_Exam/requirements.txt
import numpy as np
import pandas as pd
import torch
import datasets
import evaluate
import seaborn as sns
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sentence_transformers import SentenceTransformer

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 47.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 47.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 59.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 MB 62.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 39.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 57.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 95.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62

2024-12-21 11:17:46.976556: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734776266.996418     549 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734776267.002790     549 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 11:17:47.025709: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [89]:
# Load embeddings
emb_full=pd.read_csv('../Data/IMDB_Embeddings_Base.csv')
emb_full = emb_full.drop(columns=['Unnamed: 0'])

# Change rating to individual rating, to avoid underfitting.
IMDb_dataframe = pd.read_csv('/work/NLP_IMDb_Exam/Data/review_dataframe.csv')
IMDb_dataframe = IMDb_dataframe[IMDb_dataframe['rating'] != "Null"] # Remove nulls

emb_full['rating'] = IMDb_dataframe['rating']
emb_full = emb_full.dropna(subset=['rating'])

from sklearn.model_selection import train_test_split
tt_split = emb_full.drop(columns=['review'])

tt_split = train_test_split(tt_split, train_size=0.7, random_state=42)
finaleval=tt_split[1]
subset=tt_split[0]

train_features = subset.drop(columns=["rating"]).to_numpy()
train_labels = np.float32(subset['rating'].to_numpy())
eval_features = finaleval.drop(columns=["rating"]).to_numpy()
eval_labels = np.float32(finaleval[['rating']].to_numpy())

In [90]:

import tensorflow as tf
#initialize
neur = tf.keras.models.Sequential()
#layers
neur.add(tf.keras.layers.Dense(units=384, activation='relu', input_dim = 384, name="Input_Layer"))
neur.add(tf.keras.layers.Dense(units=700, activation='relu',  name = "Hidden_layer1"))
neur.add(tf.keras.layers.Dense(units=700, activation='relu',  name = "Hidden_layer2"))
neur.add(tf.keras.layers.Dense(units=700, activation='relu',  name = "Hidden_layer3"))
neur.add(tf.keras.layers.Dense(units=20, activation='sigmoid', name = "Summation_Layer"))
#output layer / no activation for output of regression
neur.add(tf.keras.layers.Dense(units=1, activation=None,))

#using mse for regression. Simple and clear
neur.compile(loss='mse', optimizer='adam', metrics=['mse'])

neur.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Layer (Dense)         (None, 384)               147840    
                                                                 
 Hidden_layer1 (Dense)       (None, 700)               269500    
                                                                 
 Hidden_layer2 (Dense)       (None, 700)               490700    
                                                                 
 Hidden_layer3 (Dense)       (None, 700)               490700    
                                                                 
 Summation_Layer (Dense)     (None, 20)                14020     
                                                                 
 dense_9 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1412781 (5.39 MB)
Trainable params: 141

In [91]:
#train
neur.fit(train_features, train_labels, batch_size=10000, epochs=30)

Epoch 1/30
53/53 [==============================] - 12s 204ms/step - loss: 24.4466 - mse: 24.4466
Epoch 2/30
53/53 [==============================] - 11s 203ms/step - loss: 17.8816 - mse: 17.8816
Epoch 3/30
53/53 [==============================] - 11s 200ms/step - loss: 14.7901 - mse: 14.7901
Epoch 4/30
53/53 [==============================] - 11s 202ms/step - loss: 12.5401 - mse: 12.5401
Epoch 5/30
53/53 [==============================] - 11s 204ms/step - loss: 10.9595 - mse: 10.9595
Epoch 6/30
53/53 [==============================] - 11s 200ms/step - loss: 9.8926 - mse: 9.8926
Epoch 7/30
53/53 [==============================] - 11s 201ms/step - loss: 9.2039 - mse: 9.2039
Epoch 8/30
53/53 [==============================] - 11s 201ms/step - loss: 8.7787 - mse: 8.7787
Epoch 9/30
53/53 [==============================] - 11s 203ms/step - loss: 8.5292 - mse: 8.5292
Epoch 10/30
53/53 [==============================] - 11s 203ms/step - loss: 8.3896 - mse: 8.3896
Epoch 11/30
53/53 [==========

In [92]:
# Predict
test_out = neur.predict(eval_features)
output = tt_split[1]
output['predicted'] = test_out
output['actual'] = eval_labels
output['difference'] = abs((output['actual']-output['predicted']))
output

7029/7029 [==============================] - 9s 1ms/step


,0,1,2,3,4,5,6,7,8,9,...,378,379,380,381,382,383,rating,predicted,actual,difference
435297,-0.046042,-0.024523,-0.008264,-0.047876,0.044498,0.031327,0.024902,0.048147,0.037168,-0.101125,...,-0.045955,-0.048650,0.100527,-0.087078,0.058416,0.004859,6,7.210439,6.0,1.210439
744156,-0.073523,-0.011871,-0.015912,0.009866,-0.009914,-0.039838,0.046886,0.007994,-0.050477,-0.108266,...,-0.083835,0.017802,0.061087,-0.002055,0.013124,0.020771,8,7.210439,8.0,0.789561
130893,0.029877,-0.057758,0.024739,-0.024524,0.078525,0.075445,0.036956,0.097149,0.050360,0.043058,...,0.004985,0.009197,0.078138,0.007179,0.068237,0.005067,10,7.210439,10.0,2.789561
647367,-0.099029,-0.023768,0.020476,-0.073793,0.025380,0.047399,0.014365,0.114303,0.024407,-0.062590,...,-0.033416,0.036509,0.075786,0.021914,-0.101253,-0.009372,6,7.210439,6.0,1.210439
77648,-0.063344,-0.047485,0.025725,-0.005244,0.073899,0.016331,-0.002938,-0.004925,-0.042526,0.041163,...,-0.015922,0.054392,0.064831,-0.062135,-0.011530,0.037574,10,7.210439,10.0,2.789561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449954,-0.065505,0.002891,-0.017705,-0.086923,0.035647,0.127568,0.009664,-0.030516,0.032349,-0.050407,...,-0.102480,0.058012,0.049230,-0.044122,-0.004591,0.041973,7,7.210439,7.0,0.210439
327153,-0.005065,-0.082990,-0.024087,-0.020606,-0.018116,0.034229,0.010700,-0.006205,-0.032512,-0.027970,...,0.006783,0.057496,-0.024980,-0.043181,-0.020225,-0.007584,1,7.210439,1.0,6.210439
35187,-0.073082,0.003002,-0.054063,0.025394,-0.067787,-0.025594,0.017119,0.013311,0.026750,-0.019315,...,-0.008887,0.086786,0.040674,0.061472,0.069417,-0.064912,10,7.210439,10.0,2.789561
614589,-0.097950,-0.045058,0.009053,-0.048344,-0.037359,0.038105,-0.040910,-0.022036,0.010389,-0.060047,...,0.021185,-0.037845,0.094376,0.031658,-0.036115,-0.008909,10,7.210439,10.0,2.789561


In [93]:
output[['predicted', 'actual']].corr()

,predicted,actual
predicted,NaN,NaN
actual,NaN,1.0
